# Get Data

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import json
import os.path

In [14]:
t_map = pd.read_csv('/content/drive/MyDrive/CSVs/OpenStreetMap_Tourist_Attractions_for_North_America.csv')

<ipython-input-14-0ee21db77ba6>:1: DtypeWarning: Columns (4,7,10,15,18,23,35) have mixed types. Specify dtype option on import or set low_memory=False.
  t_map = pd.read_csv('/content/drive/MyDrive/CSVs/OpenStreetMap_Tourist_Attractions_for_North_America.csv')


In [ ]:
bcols = ['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'is_open', 'categories']
data = []
filename = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_business.json'

with open(filename) as f:
  for line in f:
    doc = json.loads(line)
    add = []
    for bcol in bcols:
      add.append(doc[bcol])
    data.append(add)

businesses = pd.DataFrame(data = data, columns = bcols)

In [ ]:
rcols = ['review_id', 'business_id', 'stars', 'text']
data = []
filename = '/content/drive/MyDrive/yelp_dataset/yelp_academic_dataset_review.json'

with open(filename) as f:
  for line in f:
    doc = json.loads(line)
    add = []
    for rcol in rcols:
      add.append(doc[rcol])
    data.append(add)

reviews = pd.DataFrame(data = data, columns = rcols)

# Business Preprocessing

In [ ]:
#Isolate to restaurants only

businesses['categories'] = businesses['categories'].fillna('Nothing')
restaurants = businesses[businesses['categories'].str.startswith('Restaurants')]

In [ ]:
#Drop closed businesses
restaurants = restaurants[restaurants['is_open'] == 1]
restaurants = restaurants.drop('is_open', axis = 1)

In [ ]:
#Unify addresses
restaurants['address'] = restaurants['address'] + ', ' + restaurants['city'] + ', ' + restaurants['state'] + ' ' + restaurants['postal_code']

In [ ]:
#get categories
cats = restaurants['categories'].str.split(',', expand = True)

restaurants['cat_1'] = cats[1].str.strip()
restaurants['cat_2'] = cats[2].str.strip()
restaurants['cat_3'] = cats[3].str.strip()
restaurants = restaurants.drop('categories', axis = 1)

In [ ]:
restaurants = restaurants.drop(['city', 'state'], axis = 1)
restaurants = restaurants.reset_index(drop = True)
restaurants

,business_id,name,address,postal_code,latitude,longitude,stars,cat_1,cat_2,cat_3
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,"935 Race St, Philadelphia, PA 19107",19107,39.955505,-75.155564,4.0,Food,Bubble Tea,Coffee & Tea
1,9OG5YkX1g2GReZM0AskizA,Romano's Macaroni Grill,"5505 S Virginia St, Reno, NV 89502",89502,39.476117,-119.789339,2.5,Italian,None,None
2,ljxNT9p0y7YMPx0fcNBGig,Tony's Restaurant & 3rd Street Cafe,"312 Piasa St, Alton, IL 62002",62002,38.896563,-90.186203,3.0,Specialty Food,Steakhouses,Food
3,ABxoFuzZy5mqQ8C5FJJajQ,Core de Roma,"201 Jefferson St, Bala Cynwyd, PA 19004",19004,40.028357,-75.238084,5.0,Italian,None,None
4,ppFCk9aQkM338Rgwpl2F5A,Wawa,"3604 Chestnut St, Philadelphia, PA 19104",19104,39.954573,-75.194894,3.0,Automotive,Delis,Gas Stations
...,...,...,...,...,...,...,...,...,...,...
9958,wVxXRFf10zTTAs11nr4xeA,PrimoHoagies,"6024 Ridge Ave, Philadelphia, PA 19128",19128,40.032483,-75.214430,3.0,Specialty Food,Food,Sandwiches
9959,sf_oQ62L8UEnOOLf00nNGA,Pizza Hut,"5028 Old Hickory, Hermitage, TN 37076",37076,36.193201,-86.614748,3.0,Pizza,Fast Food,Chicken Wings
9960,l9eLGG9ZKpLJzboZq-9LRQ,Wawa,"19 N Bishop Ave, Clifton Heights, PA 19018",19018,39.925656,-75.310344,3.0,Sandwiches,Convenience Stores,Coffee & Tea
9961,WnT9NIzQgLlILjPT0kEcsQ,Adelita Taqueria & Restaurant,"1108 S 9th St, Philadelphia, PA 19147",19147,39.935982,-75.158665,4.5,Mexican,None,None


# Review Preprocessing

In [ ]:
#Drop reviews not about open restaurants
merged = reviews.merge(restaurants[['business_id']], on = 'business_id', how = 'right')
reviews = merged.dropna(subset = ['business_id'])

In [ ]:
reviews = reviews.reset_index()
reviews = reviews.drop('index', axis = 1)
reviews

,review_id,business_id,stars,text
0,BXQcBN0iAi1lAUxibGLFzA,MTSW4McQd7CbVtyjqoe9mw,4.0,This is nice little Chinese bakery in the hear...
1,uduvUCvi9w3T2bSGivCfXg,MTSW4McQd7CbVtyjqoe9mw,4.0,This is the bakery I usually go to in Chinatow...
2,a0vwPOqDXXZuJkbBW2356g,MTSW4McQd7CbVtyjqoe9mw,5.0,"A delightful find in Chinatown! Very clean, an..."
3,MKNp_CdR2k2202-c8GN5Dw,MTSW4McQd7CbVtyjqoe9mw,5.0,I ordered a graduation cake for my niece and i...
4,D1GisLDPe84Rrk_R4X2brQ,MTSW4McQd7CbVtyjqoe9mw,4.0,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...
...,...,...,...,...
941855,Kt3gFeW1rhZz7RuiV-6Tcw,2O2K6SXPWv56amqxCECd4w,5.0,This is my favorite food truck! I only wish I ...
941856,ruy3Ycey_gGbwkE_3TX1Fg,2O2K6SXPWv56amqxCECd4w,5.0,This food truck was stupid. Stupidly delicious...
941857,C_l8NTpvNOEUorEmEOusaA,2O2K6SXPWv56amqxCECd4w,5.0,Bubba never disappoints i go to his fb page an...
941858,q39JOIkHmIhdmYnjEhZCdQ,2O2K6SXPWv56amqxCECd4w,1.0,The truck was invited to our office for a part...


# Map Preprocessing

In [15]:
#Renaming
renames = {'X': 'longitude', 'Y': 'latitude', 'osm_id2': 'attraction_id', 'tourism': 'type', 'attraction_id': 'drop'}
t_map = t_map.rename(columns = renames)

In [16]:
#Drop stuff
to_keep = ['longitude', 'latitude', 'website', 'addr_city', 'addr_country', 'addr_housenumber', 'addr_province', 'addr_postcode', 'addr_state', 'addr_street', 'addr_unit', 'name', 'type', 'attraction_id']
t_map = t_map[to_keep]

In [17]:
#Unite addresses
feats = ['addr_housenumber', 'addr_street', 'addr_unit', 'addr_city', 'addr_state', 'addr_province', 'addr_country', 'addr_postcode']
for feat in feats:
  t_map[feat] = t_map[feat].astype(str)
  t_map[feat] = t_map[feat].replace('nan', '')

t_map['address'] = t_map['addr_housenumber'].str.strip() + ' ' + t_map['addr_street'].str.strip() + ', ' + t_map['addr_city'].str.strip() + ', ' + t_map['addr_state'].str.strip() + t_map['addr_province'].str.strip() + ' ' + t_map['addr_postcode'].str.strip() + ', ' + t_map['addr_country'].str.strip()


In [18]:
#Set unknown if any fields dropped
for index, row in t_map.iterrows():
  addr = row['address'].split(',')
  if any(field.strip() == '' for field in addr):
    t_map.at[index, 'address'] = 'Unknown'

In [ ]:
#Fill website column
t_map['website'].fillna('Unknown', inplace = True)
t_map['website']

In [20]:
#Second round drops
t_map = t_map.drop(feats, axis = 1)
t_map = t_map.dropna()

In [ ]:
t_map = t_map.reset_index(drop = True)
t_map

# Distance Stuff

In [ ]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point

In [ ]:
rest_info = restaurants[['business_id', 'longitude', 'latitude']]
t_info = t_map[['attraction_id', 'longitude', 'latitude']]

In [ ]:
def haversine(lon1, lat1, lon2, lat2):

    #Radius of earth in km
    r = 6378.137

    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    dist = r * c
    return dist

In [ ]:
close_enough = 25 #kilometers

In [ ]:
gdf_rests = gpd.GeoDataFrame(rest_info, geometry=[Point(xy) for xy in zip(rest_info.longitude, rest_info.latitude)])
gdf_tours = gpd.GeoDataFrame(t_info, geometry=[Point(xy) for xy in zip(t_info.longitude, t_info.latitude)])

gdf_rests.crs = gdf_tours.crs = "EPSG:4326"

gdf_rests = gdf_rests.to_crs("EPSG:3857")
gdf_tours = gdf_tours.to_crs("EPSG:3857")

gdf_tours_buffered = gdf_tours.copy()
gdf_tours_buffered.geometry = gdf_tours_buffered.geometry.buffer(close_enough * 1000)

close_rests = gpd.sjoin(gdf_rests, gdf_tours_buffered, op='within')

close_rests_df = pd.DataFrame(close_rests.drop(columns='geometry')).reset_index(drop=True)

In [ ]:
close_rests_df = close_rests_df.drop(columns = 'index_right')
close_rests_df['distance'] = haversine(close_rests_df['longitude_left'], close_rests_df['latitude_left'], close_rests_df['longitude_right'], close_rests_df['latitude_right'])
close_rests_df = close_rests_df.drop(columns = ['longitude_left', 'latitude_left', 'longitude_right', 'latitude_right'])
close_rests_df

,business_id,attraction_id,distance
0,MTSW4McQd7CbVtyjqoe9mw,1073691908,5.749833
1,ABxoFuzZy5mqQ8C5FJJajQ,1073691908,14.795421
2,ppFCk9aQkM338Rgwpl2F5A,1073691908,5.804820
3,3BJxm-HnvzdwD1zjmSbmyQ,1073691908,6.425897
4,bTve2mwLk5Zc01vRKqc2KQ,1073691908,4.156078
...,...,...,...
1468016,D5n8XvQqC_vzOCi63wNChQ,11099864692,10.745030
1468017,C8KFRgrMtc_naR6I7yWytg,9077244443,21.572610
1468018,C8KFRgrMtc_naR6I7yWytg,2676917087,21.296754
1468019,C8KFRgrMtc_naR6I7yWytg,2673195412,21.301340


# Download Time

In [ ]:
reviews.to_csv('reviews.csv', index = False)

In [ ]:
restaurants.to_csv('restaurants.csv', index = False)

In [ ]:
close_rests_df.to_csv('close_rests.csv', index = False)

In [ ]:
t_map.to_csv('t_map.csv', index = False)

In [ ]:
reviews.to_csv('reviews.zip', index = False, compression='gzip')

# NLP Experiment

## Model Installation

In [ ]:
%pip install transformers[sentencepiece]

In [ ]:
#Model credit to Heng Yang:
#https://huggingface.co/yangheng/deberta-v3-base-absa-v1.1

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline

absa_tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
absa_model = AutoModelForSequenceClassification \
  .from_pretrained("yangheng/deberta-v3-base-absa-v1.1")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/372 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/18.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:550: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/738M [00:00<?, ?B/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model_name = "yangheng/deberta-v3-base-absa-v1.1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

classifier = pipeline("text-classification", model=model, tokenizer=tokenizer)

## Review Sampling Setup

In [ ]:
k = 25
mini = 300

tester = reviews.groupby('business_id', sort = False)
paired = reviews.groupby('business_id', sort = False).apply(lambda x: x.sample(n = min(k, len(x)))).reset_index(drop = True)
groups = paired.groupby('business_id', sort = False)
partition = [group for _, group in groups]

In [ ]:
def select_reasonable(lists, refer):
    keyed_lists = {}
    for l in lists:
        business_id = l['business_id'].iloc[0]
        if len(l) != 0 and len(refer.get_group(business_id)) >= mini:
            keyed_lists[business_id] = l
        else:
            keyed_lists[business_id] = None
    return keyed_lists

In [ ]:
usable = select_reasonable(partition, tester)

In [ ]:
len(usable)

9963

In [ ]:
len([ls for ls in usable.keys() if usable[ls] is not None])

677

## Use Model

In [ ]:
from tqdm import tqdm

In [ ]:
#Scoring parameters
attrs = ['food', 'drink', 'service', 'value']
results = {key: [] for key in attrs}

In [ ]:
#Define scoring metric based on predicted valence and confidence score
def adjust_score(d):
    if d['label'] == 'Positive':
        return d['score']
    elif d['label'] == 'Negative':
        return -d['score']
    else:
        return 0

In [ ]:
def score_grouped_small(idx, aspect):
  bid = restaurants['business_id'].iloc[idx]
  revs = usable[bid]
  if revs is not None:
    dicts = classifier(revs['text'].tolist(),  text_pair = aspect)
    labels = [adjust_score(d) for d in dicts]
    results[aspect].append(sum(labels))
  else:
    results[aspect].append(None)

In [ ]:
#Calculate + store score
#Switch aspect as needed

for i in tqdm(range(len(restaurants))):
    score_grouped_small(i, 'drink')

100%|██████████| 9963/9963 [1:23:32<00:00,  1.99it/s]


In [ ]:
drink_scores = pd.DataFrame(results['drink'], columns = ['drink'])
drink_scores.to_csv('results_drink.csv', index = False)

In [ ]:
value_scores = pd.DataFrame(results['value'], columns = ['value'])
value_scores.to_csv('results_value.csv', index = False)

In [ ]:
food_scores = pd.DataFrame(results['food'], columns = ['food'])
food_scores.to_csv('results_food.csv', index = False)

In [ ]:
service_scores = pd.DataFrame(results['service'], columns = ['service'])
service_scores.to_csv('results_service.csv', index = False)

# Finishing Up

In [ ]:
drinks = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_drink.csv')
values = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_value.csv')
foods = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_food.csv')
services = pd.read_csv('/content/drive/MyDrive/Scoresheets/results_service.csv')

In [ ]:
total = pd.concat([drinks, values, foods, services], axis = 1)
total

In [ ]:
total.to_csv('scores.csv', index = False)